In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from statistics import stdev, variance, mean
from fastai.tabular.all import *
from fastai.collab import CollabDataLoaders, collab_learner
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)

In [ ]:
# Import of taining and test data
train = pd.read_csv("dados/train.csv")
test = pd. read_csv("dados/test.csv")
jokes = pd.read_csv("dados/jokes.csv")
print("Train " + str(train.shape))
print("Test " + str(test.shape))

In [ ]:
train

In [ ]:
test

In [ ]:
jokes

In [ ]:
train["joke_text"] = train.joke_id.apply(lambda x: jokes.loc[jokes.joke_id == x, "joke_text"][x-1])
test["joke_text"] = test.joke_id.apply(lambda x: jokes.loc[jokes.joke_id == x, "joke_text"][x-1])

In [ ]:
train.info()

In [ ]:
# Cheching uniqueness of data
train.select_dtypes(["int64", "float64", "object"]).nunique().sort_values().to_frame().T

In [ ]:
plt.figure(figsize=(6, 4))
plt.hist(train.Rating, edgecolor="black")
plt.title("Histogram", fontweight="bold", fontsize=14)
plt.xlabel("Rating")
plt.ylabel("Probability")
plt.axvline(train.Rating.mean(), color="red", linestyle="--")
plt.axvline(train.Rating.median(), color="blue", linestyle="--")
plt.legend(title="", labels=["Média","Mediana"], ncol=1, loc="best")
plt.show()

In [ ]:
mean(train.Rating), stdev(train.Rating), variance(train.Rating)

In [ ]:
dls = CollabDataLoaders.from_df(
    train, 
    user_name="user_id", 
    item_name="joke_id", 
    rating_name="Rating", 
    bs=2000,
    valid_pct=0.2, 
    shuffle_train=True,
    seed=2
)

dls.show_batch()

In [ ]:
class RMSELoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss()
        
    def forward(self, yhat, y):
        return torch.sqrt(self.mse(yhat, y))
    
def RMSELossFlat(*args, axis=-1, floatify=True, **kwargs):
    return BaseLoss(RMSELoss, *args, axis=axis, floatify=floatify, is_2d=False, **kwargs)

In [ ]:
model = collab_learner(dls, use_nn=False, y_range=(-10. ,10.), n_factors=250, loss_func=RMSELossFlat())
model.lr_find()

In [ ]:
model.fit_one_cycle(n_epoch=35, lr_max=10e-2, wd=0.3)

In [ ]:
model.show_results()

In [ ]:
pp = model.dls.test_dl(test)
preds, _ = model.get_preds(dl=pp)

In [ ]:
test["Rating"] = preds
test.loc[:, ["id", "Rating"]].to_csv("submission.csv", index=False)
test.head()

In [6]:
scor = 4.0110398789 - 3.9912059635
round(scor, 10)

0.0198339154